In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path
from dotenv import load_dotenv

from src.metrics import pearson_metric
from src.feature_engineering import features_pca

load_dotenv()

True

In [ ]:
dataset_dir = Path(os.environ['dataset_dir'])
full_data = pd.read_feather(dataset_dir / 'train32.feather')
sample_data = pd.read_csv(dataset_dir / 'train_sample.csv', index_col=0)

In [ ]:
data = full_data

In [ ]:
# data = data.set_index('row_id')

In [ ]:
# train on earlier data, test on later data
train = data[data.time_id < 1000]
test = data.query("1000 <= time_id")

x_train, y_train = train.drop('target', axis=1), train[['time_id', 'target']]
x_test, y_test = test.drop('target', axis=1), test[['time_id', 'target']]

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
N_INVESTMENTS = 3579
tss = TimeSeriesSplit(n_splits=2, max_train_size=1772778, gap=N_INVESTMENTS)

In [ ]:
from autosklearn.regression import AutoSklearnRegressor
model = AutoSklearnRegressor(
    time_left_for_this_task=150*3600,
    per_run_time_limit=2400,
    memory_limit=300*1024,
    resampling_strategy=tss,
)

In [ ]:
model.fit(x_train, y_train.target)

In [12]:
print(model.leaderboard())

          rank  ensemble_weight            type      cost     duration
model_id                                                              
335          1             0.14   liblinear_svr  0.988336  1598.138604
370          2             0.10   liblinear_svr  0.988367  1419.799253
163          3             0.10             sgd  0.989029   719.830871
219          4             0.04  ard_regression  0.989237   331.501770
350          5             0.08   liblinear_svr  0.990235   357.218997
379          6             0.02  ard_regression  0.990278   774.472420
43           7             0.04   liblinear_svr  0.994438  1724.092822
169          8             0.04   liblinear_svr  0.994505   357.787291
135          9             0.02   liblinear_svr  0.996758  1859.565843
37          10             0.02   decision_tree  0.997795   262.276921
224         11             0.02   decision_tree  0.997795   636.084652
226         12             0.02   decision_tree  0.997795  1755.752579
111   

In [ ]:
model.refit(x_train, y_train.target)
refit_y_pred = model.predict(x_test)

In [13]:
y_pred = pd.DataFrame({'target': refit_y_pred}, index=x_test.index)
rmse = np.mean(np.sqrt((y_pred.target - y_test.target) ** 2))
pearson_metric(y_test, y_pred)

0.11552864162196788